# Dependencies

In [1]:
from sys import path
path.append('../../src/')

import cv2
import pickle as pkl
from os.path import join, exists
from sys import stdout

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from icao_dataset import ICAODataset
from utils import load
from data_structures import Point, Rect

In [2]:
PATH_TO_ROOT = "../.."

FILE_DATASET = join(PATH_TO_ROOT, 'data/cropped_faces/dataset.pkl')

print(FILE_DATASET)

../..\data/cropped_faces/dataset.pkl


# Data Loading 

In [3]:
dataset = pkl.load(open(FILE_DATASET, "rb"))

In [4]:
def get_patch(im, size=160):
    half_size = size // 2
    im_h, im_w = im.shape[:2]
    center_face = Point(im_w // 2, im_h // 2)
    
    left = abs(min(0, center_face.x - half_size))
    top = abs(min(0, center_face.y - half_size))
    right = max(0, (center_face.x + half_size) - im_w)
    bottom = max(0, (center_face.y + half_size) - im_h)
    
    im_res = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)

    random = lambda: np.random.choice(half_size)
    left = max(0, center_face.x - random())
    top = max(0, center_face.y - random())
    right = left + size
    bottom = top + size
    
    return im_res[top:bottom, left:right]

In [5]:
N_FILES = len(dataset.image_files)
N_PATCH_NON_COMPLIANT = 10

new_dataset = ICAODataset()
for i, (file, _, mrk) in enumerate(dataset):
    print(f"{i + 1} of {N_FILES}", end="\r")
    stdout.flush()
    
    im = cv2.imread(join("..", file), cv2.IMREAD_COLOR)
    if len(im) == 0:
        continue

    is_compliant = mrk.photo_reqs.pixelation.is_compliant()
    n_patches = 1 if is_compliant else N_PATCH_NON_COMPLIANT
    list_patches = [get_patch(im) for _ in range(n_patches)]

    for patch in list_patches:
        if len(patch) == 0 or patch.shape != (160, 160, 3):
            continue
        new_dataset.image_files.append(file)
        new_dataset.images.append(patch)
        new_dataset.mrks.append(mrk)

In [6]:
dist = [mrk.photo_reqs.pixelation.value for _, _, mrk in new_dataset]
np.unique(dist, return_counts=True)

(array([0, 1]), array([3700, 5360], dtype=int64))

In [7]:
FILE_DATASET = join(PATH_TO_ROOT, "data/pixelation/dataset.pkl")
pkl.dump(new_dataset, open(FILE_DATASET, "wb"), protocol=4)

In [ ]:
new_dataset.save(
    folder_images=join(PATH_TO_ROOT, "data/pixelation/images"),
    folder_mrks=join(PATH_TO_ROOT, "data/pixelation/ground_truth"),
)